<a href="https://colab.research.google.com/github/parksewook/data/blob/main/%EB%8C%80%EC%A0%84%EA%B4%91%EC%97%AD%EC%8B%9C_%EB%8B%B9%EC%A7%81%EC%9D%98%EB%A3%8C%EA%B8%B0%EA%B4%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **대전광역시 당직의료기관 정보를 조회하고 지도에 매핑하는 프로그램**


### 1. 공공데이터포털에서 대전광역시 당직의료기관정보 api를 요청해서 표의 형태로 1차로 시각화 하기

In [ ]:
# 필요한 라이브러리 설치
import json
import pandas as pd
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, quote
from pandas.io.json import json_normalize

# 원하는 기간의 의료기관 당직일 입력
searchkeyword = input("조회를 원하는 의료기관 당직일을 입력해 주세요\nYYYY-MM-DD : ")

# 대전광역시 당직의료기관 공공 api를 json형식으로 요청 (요청메세지의 검색(당직일)
# (searchKeyword)은 앞에서 입력) (출처 : 공공데이터포털 )
url = ("http://apis.data.go.kr/6300000/hospitalDataService/hospitalDataListJson?serviceKey=sgqqn0iw%2BOkWOrg1ZV5X6mVHgaFoGbXsbLEwh2p3bKPZhWLXKDPdR%2F49%2FtVt%2F3XQbRur9ZiwPomz4Md%2B%2BZ0FVg%3D%3D&numOfRows=10&pageNo=1&searchKeyword=" + searchkeyword)
response = urlopen(url)
# 요청한 json파일을 utf-8형식으로 디코딩하여 저장
json_api = response.read().decode("utf-8")
json_file = json.loads(json_api)

# 호출하여 저장한 json데이터의 msgBody부분만 데이터 프레임으로 저장
df = pd.DataFrame(json_file['msgBody'])
# 데이터 프레임의 열이름 변경후 1차 표의 형태로 시각화
df.rename(columns={'addr1':'주소', 'hospitalSeq':'기관일련번호', 'nm':'기관이름', 'phone':'기관전화번호', 'section':'기관구분', 'etc':'응급실 유무'}, inplace=True)
df

조회를 원하는 의료기관 당직일을 입력해 주세요
YYYY-MM-DD : 2021-12-17


,기관일련번호,기관이름,주소,기관전화번호,기관구분,응급실 유무
0,1,유항외과의원,서구 계룡로 646 (용문동),042-536-2800,의원,N
1,13,조은정형외과의원,대덕구 계족산로 2 (중리동),042-639-7585,의원,N
2,33,한사랑의원 (둔산동),서구 문예로 63 (둔산동),042-486-7577,의원,N
3,38,순풍산부인과내과의원/금(18시까지),중구 계백로 1560 (유천동),042-582-7591,의원,N
4,42,오정성모의원,대덕구 대전로 1121 (오정동),042-636-0070,의원,N
5,46,유성연세의원,유성구 계룡로 48 (봉명동),042-822-0456,의원,N
6,57,연합소아청소년과,중구 계백로 1716번길 26 (문화동),042-221-7533,의원,N
7,154,더블유여성병원,서구 문정로48번길 70 (탄방동),042-485-7777,병원(산부인과),N
8,162,탄방엠블병원(주말 18시까지),대전광역시 서구 문정로 48번길 70 (탄방동),00-1899-3275,병원,N
9,171,김영소아청소년과의원,동구 은어송로 63,042-272-5119,의원,N


### 2. 당직의료기관의 매핑을 위해, 앞서 만들어진 데이터의 전처리 및 vworld geocoding api를 사용해 '주소' 데이터를 '좌표값'으로 geocoding 하기

In [ ]:
# 앞서 전처리한 데이터에서 '주소'데이터만 추출해 list형태로 변환
addr = df['주소']
addr_list = addr.values.tolist()
# 변화한 위도, 경보 데이터를 넣기위한 빈 list 생성 
lis_lat = []
lis_lng = []

# vworld api 어플리케이션 key
vworld_apikey = "73D1B383-F544-3D19-90BB-9EFA4F7C42FE"


for addr2 in addr_list:
  
  # vworld의 geocoding api 호출
  geourl = "http://api.vworld.kr/req/address?service=address&request=getCoord&type=ROAD&refine=false&key=%s&" \ 
   % (vworld_apikey) + urlencode({quote_plus('address'):addr2}, encoding='UTF-8')

  request = Request(geourl)
  response = urlopen(request)
  rescode = response.getcode()

  if rescode == 200:
    response_body = response.read().decode('utf-8')
  else:
    print('error code:', rescode)

  # 호출된 데이터에서 y와 x의 값만 list에 저장
  jsonData = json.loads(response_body)
  lat = float(jsonData['response']['result']['point']['y'])
  lis_lat.append(lat)
  lng = float(jsonData['response']['result']['point']['x'])
  lis_lng.append(lng)

# 위도, 경도 두 list를 dataframe로 변환후 df의 기관이름 열까지 지도 매핑을 위한 데이터 정제
df_lat = pd.DataFrame(lis_lat, columns=['y'])
df_lng = pd.DataFrame(lis_lng, columns=['x'])
df_point = pd.concat([df['기관이름'], df_lat, df_lng], axis=1)
df_point

SyntaxError: ignored

### 3. geocoding으로 구해진 좌표값과 '기관이름' 데이터를 이용해 지도 매핑과 툴팁으로 2차 시각화

In [ ]:
import folium

# 검색기간의 모든 당직의료기관 중심으로 맵 초기 카메라 위치 설정
lat = df_point['y'].mean()
lng = df_point['x'].mean()

hospitalMap = folium.Map([lat, lng], zoom_start=12)

# 정제된 위도와 경도로 당직의료기관 매핑
for i in df_point.index:
  sub_lat = df_point.loc[i,'y']
  sub_lng = df_point.loc[i, 'x']

  title = df_point.loc[i,'기관이름']

  folium.Marker([sub_lat, sub_lng], tooltip = title).add_to(hospitalMap)

hospitalMap